In [41]:
#!pip install qpsolvers
#!pip install cvxopt

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg
from tqdm import tqdm
import numba 
from numba import njit, vectorize, jit
import time 
import scipy

import qpsolvers
from qpsolvers import solve_qp
from qpsolvers import dense_solvers, sparse_solvers

from scipy import optimize

import cvxopt
import cvxopt.solvers

In [2]:
X_train = pd.read_csv('data/Xtr0_mat100.csv',delimiter= ' ', header= None)
Y_train = pd.read_csv('data/Ytr0.csv',delimiter= ',')
X_test = pd.read_csv('data/Xte0_mat100.csv',delimiter= ' ', header= None)

In [3]:
X = X_train.values
Y = Y_train['Bound'].to_numpy()

In [4]:
print(Y.shape)

(2000,)


In [87]:
@njit
def GaussianKernel(x, y, sig2 = 1): 
    return np.exp(-numpy.linalg.norm(x-y)**2/(2*sig2))

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=5):
    return (1 + np.dot(x, y)) ** p

In [6]:
time1 = time.time()
Kernel = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : GaussianKernel(x2, x1), 1, X), 1, X)
time2 = time.time()
print(time2 - time1, 's')

18.203829526901245 s


In [10]:
print(Kernel)

x_train = X[:1900]
y_train = Y[:1900]
x_val = X[1900:]
y_val = Y[1900:]

[[1.         0.98475818 0.98371162 ... 0.98848832 0.98942306 0.98895558]
 [0.98475818 1.         0.9799995  ... 0.98545651 0.98173781 0.97988373]
 [0.98371162 0.9799995  1.         ... 0.97826428 0.98196982 0.97699373]
 ...
 [0.98848832 0.98545651 0.97826428 ... 1.         0.98708787 0.98545651]
 [0.98942306 0.98173781 0.98196982 ... 0.98708787 1.         0.98650493]
 [0.98895558 0.97988373 0.97699373 ... 0.98545651 0.98650493 1.        ]]


J'ai fais 2 SVM avec deux technique différentes, et ils donnent des résultats différent, SVM_1 j'ai remplacé alpha*Y pas beta et minimisé par rapport à beta, et SVM_1 c'est le normal comme la formule du cours. Je pense que SVM_2 a des résultats plus logiques, faut prendre C=5 environ.

Au début j'ai voulu faire la méthode du cours mais rien marchait, je me suis donc inspiré d'un truc sur github où ils utilisaient le changement de variable avec beta donc j'ai fais ça, ce qui marche mais semble étrange. Après j'ai réessayé la méthode du cours (SVM_2) et ça fonctionne aussi.

In [133]:
#avec cvxopt pour résoudre min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C  
#(en gros b= alpha * diag(Y))
class SVM_1:
    def __init__(self, kernel = GaussianKernel, C = 1):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        #calculate the kernel
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
        
        n = len(Y)
        lbd = 1
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        #take Y as -1 and 1
        label = 2 * Y - 1
        
        
        P = cvxopt.matrix(np.outer(label, label) * K ) 
        q = cvxopt.matrix(-np.ones(n))
        A = cvxopt.matrix(label, (1,n), 'd')
        b = cvxopt.matrix(0.0)
                
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
        '''
        # b <= C
        G1 = np.eye(n)
        h1 = np.ones(n) * self.C
        
        # -b <= 0
        G2 = -np.eye(n)
        h2 = np.zeros(n)
        
        G = cvxopt.matrix(np.vstack((G2, G1)))
        h = cvxopt.matrix(np.hstack((h2, h1)))

        #min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C
        solver = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        self.alpha = np.ravel(solver['x'])
        
        #Je retire les vecteurs avec un alpha trop petit
        eps = 1e-5
        supportIndices = self.alpha > eps
        ind = np.arange(n)[supportIndices]
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha_ = self.alpha[supportIndices]  #alpha_ : alpha sans les alpha < eps
        
        #Bias
        self.b = 0
        for i in range(len(self.alpha_)):
            self.b = self.support_Y[i]
            self.b -= np.sum( self.alpha_ * self.support_Y * K[ind[i], supportIndices])
        self.b /= len(self.alpha_)
    
    def predict(self, X):
        
        y_predict = np.zeros(len(X))
        
        for i in range(len(X)):
            for alpha, sv, label in zip(self.alpha_, self.support_vectors, self.support_Y):
                y_predict[i] += alpha * label * self.kernel(sv, X[i]) 
        
        return ((y_predict + self.b) > 0)*1
        #return y_predict + self.b

In [153]:
#avec cvxopt pour résoudre min_a 1/2 * alpha.T * K * alpha - alpha.T * Y s.t. 0<=y*alpha<=C
class SVM_2:
    def __init__(self, kernel = GaussianKernel, C = 1):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
        
        n = len(Y)
        #lbd = 1
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        label = 2 * Y - 1
        
        # P=K et q=-Y
        P = cvxopt.matrix(K) 
        q = cvxopt.matrix(-label, tc='d')
                
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        '''
        
        # Condition 0 <= alpha_i * Y_i <= C
        G1 = np.diag(-label)
        G2 = np.diag(label)
        G = cvxopt.matrix(np.vstack((G1, G2)), tc='d')
        
        h1 = np.zeros((n, 1), dtype='float64')
        h2 = self.C * np.ones((n, 1), dtype='float64')
        h = cvxopt.matrix(np.vstack((h1, h2)))

        # solves min_a 1/2 a^T * P * a + q^T * a s.t. G*a <= h
        solver = cvxopt.solvers.qp(P, q, G, h)
        
        self.alpha = np.ravel(solver['x'])
        
        #Je retire les vecteurs avec un alpha trop petit
        eps = 1e-5
        supportIndices = np.abs(self.alpha) > eps
        ind = np.arange(n)[supportIndices]
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha_ = self.alpha[supportIndices]  #alpha_ : alpha sans les alpha < eps
        
        #Bias
        self.b = 0
        for i in range(len(self.alpha_)):
            self.b = self.support_Y[i]
            self.b -= np.sum( self.alpha_ * K[ind[i], supportIndices])
        self.b /= len(self.alpha_)
    
    def predict(self, X):
        
        y_predict = np.zeros(len(X))
        
        for i in range(len(X)):
            for j in range(len(self.alpha_)):
                y_predict[i] += self.alpha_[j] * self.kernel(self.support_vectors[j], X[i])
        
        return ((y_predict + self.b) > 0)*1
        #return y_predict + self.b
        

In [158]:
model = SVM_2(kernel = GaussianKernel, C=5)

model.fit(x_train, y_train)

results = model.predict(x_val)

In [159]:
results = model.predict(x_val)
print( results, y_val)
n = results == y_val
print(sum(n))
print(model.alpha, len(model.alpha_), model.b)


[1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0
 0 1 1 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0] [1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0
 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1
 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
56
[-0.99999995  0.99999995  0.99999996 ... -0.9999999  -0.99999994
  0.99999995] 1809 0.0007539632839989708


EN dessous il y a un autre avec la même méthode que 'SVM_1' mais avec un autre optimiseur.

In [160]:
#Avec qp_solver pour résoudre min_b 1/2 * b.T * diag(Y) * K * diag(Y) * b - b.T * 1 s.t. 0<= b <= C  
#(en gros b= alpha * diag(Y))

#Il marche et donne le même résultat que cvxopt

class SVM_autre:
    def __init__(self, kernel=GaussianKernel, C=1):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_Y = None
        
        
    def fit(self, X, Y):
        
        K = np.apply_along_axis(lambda x1: np.apply_along_axis( lambda x2 : self.kernel(x2, x1), 1, X), 1, X)
        
        n = len(Y)
        lbd = 1
        #C = 1 / (2 * n * lbd)  #ça dépend si on veut gérer C ou lambda

        label = 2 * Y - 1
        
        P = np.outer(label, label) * K
        q = - np.ones(n)
        
        '''Je réécris l'inégalité : 0<=y_i*alpha_i<=C avec C = 1/(2*lambda*n)
        comme: G*alpha<=h avec G=stack(diag(Y),-diag(Y)) et h= [C, ..., C, 0, ..., 0] (n fois C et n fois 0)
        ça revient au même et je crois que le solver devrait fonctionner avec ça, mais j'y arrive pas encore
        '''
        G = np.vstack((np.eye(n), -np.eye(n)))

        h = np.ones(2*n)
        h[:n] = h[:n] * self.C
        h[n:] = h[n:] * 0

        A = label
        b = np.array([0.])

        self.alpha = solve_qp(P, q.astype('double'), G.astype('double'), h, A, b, solver = 'quadprog')
        
        #Pour le moment je garde tout X mais faudrait retirer les X dont le alpha est trop bas
        eps = 1e-15
        supportIndices = self.alpha > eps
        ind = np.arange(n)[supportIndices]
        
        self.support_vectors = X[supportIndices]
        self.support_Y = label[supportIndices]
        self.alpha_ = self.alpha[supportIndices]
        
        #Bias
        self.b = 0
        for i in range(len(self.alpha_)):
            self.b = self.support_Y[i]
            self.b -= np.sum( self.alpha_ * self.support_Y * K[ind[i], supportIndices])
        self.b /= len(self.alpha_)
    
    def predict(self, X):

        y_predict = np.zeros(len(X))

        for i in range(len(X)):
            for alpha, sv, label in zip(self.alpha_, self.support_vectors, self.support_Y):
                y_predict[i] += alpha * label * self.kernel(sv, X[i]) 

        return ((y_predict + self.b) > 0)*1
    
    def predict_bis(self, X):
        
        def predict_one(x):
            pred = np.apply_along_axis(lambda s: self.kernel(s, x), 1, self.support_vectors)
            pred = pred * self.support_Y * self.alpha
            return np.sum(pred)
        
        preds = np.apply_along_axis(predict_one, 1, X)
        return 1 * (preds > 0)
        
        

In [95]:

svm = SVM_autre(kernel = GaussianKernel)
svm.fit(x_train, y_train)


In [96]:
results = svm.predict(x_val)
print(results, y_val)
print(svm.alpha_)

[1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0] [1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0
 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 1
 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
[1. 1. 1. ... 1. 1. 1.]


Aide svm github : https://github.com/zongmianli/mva-kernel-methods/blob/kernel-challenge/svm.py

In [132]:
#Tests sur les fct de prédiction

def predict_one(x):
    pred = np.apply_along_axis(lambda s: GaussianKernel(s, x), 1, X_bis)
    pred = pred * Y_bis * model.alpha
    return np.sum(pred)

def f_from_alpha(alpha, Kernel, X):
    return  lambda x : np.sum([alpha[i]*Kernel(X[i,:],x) for i in range(X.shape[0])])
f_alpha = f_from_alpha(model.alpha, GaussianKernel, x_train)

res = [f_alpha(x_val[i]) for i in range(len(y_val))]

preds = np.apply_along_axis(predict_one, 1, x_val)
#preds = 1 * (preds > 0)
print(preds, y_val, res)

[ 0.02414692  0.00498681  0.01534296  0.03327065  0.02499289  0.01782278
  0.00641423  0.01737927  0.00571243  0.03785679  0.02937389 -0.0020743
  0.01801311  0.01202195  0.02616614  0.02493651  0.02069579  0.00564455
 -0.00066408  0.02225113  0.00237909  0.03542867  0.00787691  0.02247312
  0.02751154  0.00633758  0.01760011  0.0165434   0.0267509   0.03206684
  0.01438477  0.03242849  0.01270201  0.01455076  0.00451208  0.01183673
  0.01354231  0.00326173  0.04398204  0.02790272 -0.00445432  0.04707981
  0.00802263  0.02043321  0.00878177  0.01557869  0.03094886  0.03406918
  0.02125684  0.02878366  0.00324311  0.01825782  0.02390436  0.00269406
  0.02602237  0.01234005  0.00085998  0.0158754   0.02489576  0.02026007
  0.00903557  0.0151773   0.03625054  0.01268538  0.00022868  0.00617082
  0.0134089   0.01244844 -0.00563212  0.00012831  0.02412231  0.02177587
  0.01969552  0.03197964  0.01770224  0.01614737  0.01016502  0.03356141
  0.00210028  0.02871426  0.00812573 -0.00372503  0.